In [1]:
# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash
# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Updated code in the notebook
username = "aacuser"
password = "password"
host = "nv-desktop-services.apporto.com"
port = 32157
db = "AAC"
collection = "animals"

# Import AnimalShelter and reload the module if needed
import importlib
import animal_shelter
from animal_shelter import AnimalShelter
importlib.reload(animal_shelter)

# Create an instance of AnimalShelter with the updated constructor
shelter = AnimalShelter(username, password, host, port, db, collection)

# Retrieve all data from the database
data = shelter.read({})  # Read data from the database

# Check if the data is None or empty, and handle accordingly
if not data:
    df = pd.DataFrame()  # Create an empty DataFrame if no data is returned
else:
    df = pd.DataFrame.from_records(data)

# Remove the '_id' column to avoid issues with Dash DataTable
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Dashboard Layout
app = JupyterDash('Austin Animal Center')

# Updated dashboard layout with reset button that refreshes the page
# Dashboard Layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=True),  # For handling page refresh
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Add a title image
    html.Center(html.Img(src='https://sarah-deleppo.com/assets/img/portfolio/animal-shelter.png', style={'height': '100px'})),  # Fixed comma

    html.Hr(),  # Horizontal rule

    # Buttons for filtering data
    html.Div([
        html.Button('Water Rescue', id='water-rescue-button', n_clicks=0, style={
            'backgroundColor': '#007BFF', 'color': 'white', 'padding': '10px 20px', 'margin-right': '10px'}),
        html.Button('Mountain/Wilderness Rescue', id='mountain-rescue-button', n_clicks=0, style={
            'backgroundColor': '#28A745', 'color': 'white', 'padding': '10px 20px', 'margin-right': '10px'}),
        html.Button('Disaster/Individual Tracking', id='disaster-tracking-button', n_clicks=0, style={
            'backgroundColor': '#FFC107', 'color': 'white', 'padding': '10px 20px', 'margin-right': '10px'}),
        html.Button('Reset Filters', id='reset-filters-button', n_clicks=0, style={
            'backgroundColor': '#DC3545', 'color': 'white', 'padding': '10px 20px'})
    ]),

    html.Br(),
    html.Br(),

    # Interactive DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'overflow': 'hidden', 'textOverflow': 'ellipsis',
        },
    ),

    html.Br(),
    html.Hr(),

    # Geolocation chart
    html.Div(id='map-id', className='col s12 m6', style={'width': '100%'}),

    html.Br(),

    # Larger pie chart below the map
    dcc.Graph(id='breed-pie-chart', style={'width': '100%', 'height': '700px'}),  # Increased height for the pie chart
])

@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', 'children'),
     Output('breed-pie-chart', 'figure')],
    [Input('water-rescue-button', 'n_clicks'),
     Input('mountain-rescue-button', 'n_clicks'),
     Input('disaster-tracking-button', 'n_clicks'),
     Input('reset-filters-button', 'n_clicks')]
)
def filter_data(water_clicks, mountain_clicks, disaster_clicks, reset_clicks):
    ctx = dash.callback_context

    # Default to all data unless a button is clicked
    filtered_df = df

    if ctx.triggered:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'reset-filters-button':
            filtered_df = df  # Reset to full data
        elif button_id == 'water-rescue-button':
            filtered_df = df[df['breed'].str.contains('siamese mix', case=False, na=False)]
        elif button_id == 'mountain-rescue-button':
            filtered_df = df[
                df['breed'].str.contains('longhair|short hair', case=False, na=False)
            ]
        elif button_id == 'disaster-tracking-button':
            # Filter for 'bat', 'raccoon', or 'chicken' in the breed column (case-insensitive)
            filtered_df = df[
                df['breed'].str.contains('bat|raccoon|chicken', case=False, na=False)
            ]

    # DataTable Update
    data = filtered_df.to_dict('records')

    # Map Update
    if not filtered_df.empty:
        lat, lon = filtered_df.iloc[0]['location_lat'], filtered_df.iloc[0]['location_long']
        updated_map = [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=12, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(filtered_df.iloc[0]['breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(filtered_df.iloc[0]['name'])
                    ])
                ])
            ])
        ]
    else:
        updated_map = [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])]

    # Pie Chart Update
    if not filtered_df.empty:
        breed_counts = filtered_df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        updated_pie_chart = px.pie(
            breed_counts, values='count', names='breed', title='Distribution of Animals by Breed', hole=0.3
        )
    else:
        updated_pie_chart = px.pie(
            values=[1], names=['No Data'], title='No Data Available'
        )

    return data, updated_map, updated_pie_chart

# Run the server
app.run_server(debug=True)



Dash app running on http://127.0.0.1:25998/
